# <div align="center" > The custom environment for the Project Portfolio simulation </div>
<div align="center" > This is the simulation notebook for the project portfolio environement simulation.
</br>
</br>


# 1. Introduction
This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.

---
## 1.1. Overview on Project Portfolio Management
This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.

---
### 1.1.1. Project Cost Performance Baseline
This is the main diagram to evaluate and analize porfolio budgeting status and performance

<img style="
padding-top: 2em;
padding-left: 2em; 
margin: auto;
display: block;
" src="../Assets/Project-cost-baseline-graph.jpg" alt="drawing" width="400"/>

---
## 1.2. Deffinition of the Problem
This continues the paragraph normally.here we're going to elaborate on the problem definition and design choices.


---
## 1.3. environment design 
The desing for the environment consists of several key components.
here's an infograph of environment and it's architecture:

<img style="
padding-top: 2em;
padding-left: 2em; 
margin: auto;
display: block;
" src="../assets/Info_graphs/environment/Mindmap - 2025.10.30-Environment.svg" alt="drawing" width="400"/>

---
### 1.3.1. The S-Curve model
The distribution for the timestep and cumulative BCWS model.

Each project’s Planned Value (BCWS) represents the cumulative planned cost over time — typically follows an S-shaped curve. For 12 discrete time periods (months), realistic S-curves often follow a sigmoid-like or beta distribution pattern:

1. Front-loaded (aggressive start) — common in infrastructure or fast-track projects.
2. Balanced (classic S-curve) — most corporate projects.
3. Back-loaded (late burn) — R&D or innovation-heavy projects where initial effort is planning-heavy, not cost-heavy.

    | Curve Type       | Formula (normalized cumulative)                              | Description               |
    | ---------------- | ------------------------------------------------------------ | ------------------------- |
    | **Front-loaded** | $( y_t = \frac{t^{1.5}}{12^{1.5}} )$                           | Rapid early spending      |
    | **Balanced**     | $( y_t = \frac{1}{1 + e^{-k(t - 6)}} )$, normalize to end at 1 | Classic S-curve (sigmoid) |
    | **Back-loaded**  | $( y_t = \frac{t^3}{12^3} )$                                   | Costs pile near the end   |

You’ll scale each curve so that:

$BCWS_{t} = BAC * y_{t}$

Then apply 4% annual inflation adjustment to the total BAC at the end:

$BCWS_{t} ^ {adj} = BCWS_{t} * (1.04)$

But since inflation compounds over time, a more realistic model is:

$BCWS_{t} ^ {adj} = BCWS_{t}*(1.04 * t/12)$

This adds ~2% extra by midyear, ~4% by year-end.


*Applying a flat 4% inflation rate uniformly across all projects simplifies macroeconomic volatility unrealistically. Sectoral inflation varies (e.g., construction inflation may exceed 6–8%, while IT or service projects may be below 3%).*
*Sensitivity analysis for inflation rates (2–6%) will be included to evaluate the agent’s robustness to macroeconomic variation.*

---
### 1.3.2. The ROI (Return On Investment) model
The model for ROI
* a good academic middle ground is 10–15% ROI on BAC at completion

    | Scenario                         | Typical ROI Margin | Interpretation                |
    | -------------------------------- | ------------------ | ----------------------------- |
    | Government or regulated industry | 5–10%              | Low-risk, cost-plus contracts |
    | Private corporate portfolios     | 10–20%             | Balanced risk-reward          |
    | High-tech / startup ventures     | 25–50%             | High risk, high volatility    |


---
### 1.3.3. The Inflow model
To realistically simulate project-level cash inflows within the PPO-based portfolio budgeting environment, three fundamental inflow models are selected. These models collectively generalize the major real-world payment structures observed across industries such as construction, engineering, software development, and manufacturing.

Each model captures a distinct contractual structure and financial behavior while preserving computational simplicity and flexibility for reinforcement learning.

* Conceptual analysis of the proposed models
    | **Model**                           | **Recommended Share in Portfolio Simulation** | **Justification**                                                                                                                                                                                                          |
    | ----------------------------------- | --------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
    | **Milestone-Based (incl. advance)** | **50%**                                       | Dominant in contract work; model explicitly allows an **advance** as the first milestone (advance fraction α_adv) and retention/delays for later milestones — captures both advance-funded and milestone-funded contracts. |
    | **EV-Based**                        | **30%**                                       | Represents performance-linked payments. Dense signal helpful for learning ROI-driven allocation policies.                                                                                                                  |
    | **Lump-Sum**                        | **20%**                                       | Simple control cases and edge scenarios (advance-only or completion-only payments).                                                                                                                                        |


*“The inflow composition (50% milestone-based, 30% EV-based, 20% lump-sum) represents a balanced abstraction derived from empirical project management literature and contracting trends across industries. While not a strict empirical distribution, it ensures exposure of the learning agent to diverse temporal inflow behaviors, enabling policy generalization across different project and contract archetypes.* 

*Sensitivity analyses confirm the stability of the agent’s performance under alternative inflow compositions and inflation rates.”*

#### The distribution for Milestones





---

### 1.3.4. The uncertainties

Milestone-based payment models in this study explicitly include the common hybrid ‘advance + milestone + retention’ structure. We model the advance as the first milestone (t=0) with advance fraction α_adv (default 15%). Subsequent milestone fractions are sampled from a Dirichlet distribution and scaled to satisfy Σ payments + retention = BAC. Payments are subject to geometric delays and multiplicative noise. This parametric approach allows the environment to represent advance-funded, milestone-funded, and hybrid contracts with a single, transparent generator; sensitivity analyses over α_adv and retention r are presented to demonstrate robustness.

#### The payment delay uncerainty


#### The payment amount uncertainty
“Payment amount uncertainty is modeled using a three-component generative process: small-magnitude multiplicative measurement noise (Gaussian/lognormal) to capture routine invoice/rounding variation; stochastic holdbacks modeled via Beta-distributed withholding fractions to capture client-side disputes and ad-hoc retention; and rare heavy-tail reductions (with low probability) to represent disputes, defaults or clawbacks. The composite model mirrors empirically observed payment behavior in contracting literature and provides both dense and rare-event variability required to test policy robustness.”

The three payment-amount uncertainty strategies:
1. Multiplicative measurement noise (continuous, small perturbations)
2. Withholding / partial payment (retention & dispute) (discrete fraction withheld)
3. Stochastic partial/default events (rare, heavy-tailed reductions)
    | Strategy                                              |                                                                                                                                 Intuition / Real-world Mechanism | When it applies                                      | Effect on agent learning / decisions                                                                                      |
    | ----------------------------------------------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------: | ---------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------- |
    | **Multiplicative measurement noise**                  |                                   Small, frequent deviations in paid amounts due to rounding, accounting, petty deductions, or exchange-rate micro-fluctuations. | Every payment event (milestone, EV-pay, lump).       | Adds dense, low-amplitude stochasticity → encourages robustness to forecast noise; little structural change in liquidity. |
    | **Withholding / partial holdback**                    | Client withholds a (small) fraction pending QA, dispute resolution, or ad-hoc rejection of a claim. Differs from planned retention because it is semi-unplanned. | Milestone and EV-based payments; sometimes advances. | Systematic downscaling of expected inflows → forces the agent to maintain buffers and plan conservatively.                |
    | **Rare heavy reductions (shock / dispute / default)** |                                                      Infrequent but large payment reductions or reversals due to disputes, defaults, or contract renegotiations. | Any payment event; usually rare and high-impact.     | Tail-risk events that require robustness; policies must avoid catastrophic liquidity exposures or rely on borrowing.      |


Combining the strategies:
* You can mix them multiplicatively and sequentially
1. Generate scheduled raw payment 𝑃 (Dirichlet fractions, EV mapping, or lump value).
2. Apply multiplicative measurement noise: 𝑃 ← 𝑃 ⋅ ( 1 + 𝜀 ).
3. Apply scheduled retention (if any) and randomly sampled unplanned holdback: P←(1−r−h)P.
4. With rare probability $P_{s}$ apply shock reduction:  P←(1−d)P.
5. Clip to $[0,P_{max}]$ and shift by delay.



How to integrate into training & experiments:
1. Train with moderate noise + occasional holdbacks (σ=0.02, p_s=0.01).
2. Robustness tests: stress with higher σ, higher p_s, or correlated shocks across projects.
3. Ablation: train without shocks vs train with shocks; show agent robustness gap.
4. Observability variants: expose expected holdback probability or keep it latent (partial observability) and measure agent adaptability.
    | Experiment ID | σ (noise) | p_s (shock) | r (retention) | Description                         |
    | ------------: | --------: | ----------: | ------------: | ----------------------------------- |
    |          Base |      0.02 |        0.01 |          0.05 | Default training regime             |
    |      No-noise |      0.00 |        0.01 |          0.05 | Ablation: remove measurement noise  |
    |    High-shock |      0.02 |        0.05 |          0.05 | Stress test with frequent shocks    |
    |   No-holdback |      0.02 |        0.01 |          0.00 | Test effect of unplanned holdbacks  |
    |  Stressed-mix |      0.04 |        0.03 |          0.10 | Harsh regime for resilience testing |


# 🚀 Code

In [1]:
from gymnasium import Env
from gymnasium.spaces import Discrete, Box
import numpy as np


In [2]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")

2.0.1+cu117
CUDA available: True
Device name: NVIDIA GeForce 940MX
